# 读取Product property


In [348]:
import re
import numpy as np
import pandas as pd


def normalize_target(s: str) -> str:
    """统一property value的符号和格式"""
    if pd.isna(s):
        return ""
    return (
        s.strip("() ").replace(",", ".")
        .replace("–", "-").replace("~", "-")
        .replace("＝", "=").replace("＝", "=")
        .replace("=<", "<=").replace("=>", ">=")
    )

def parse_lb_ub(s: str):
    """根据 Target_Value 提取 lb 和 ub"""
    s = str(s).strip()

    s = normalize_target(s)

    # 匹配 ± 格式，如 10.5 ± 2.5
    m = re.match(r"([0-9.]+)\s*[±]\s*([0-9.]+)", s)
    if m:
        center, tol = map(float, m.groups())
        return center - tol, center + tol

    # 区间格式 (a - b)
    m = re.match(r"([0-9.]+)\s*-\s*([0-9.]+)", s)
    if m:
        return float(m.group(1)), float(m.group(2))

    # 大于或等于 (≥, >=)
    m = re.search(r"[≥>]=?\s*([0-9.]+)", s)
    if m:
        return float(m.group(1)), np.nan

    # 小于或等于 (≤, <=)
    m = re.search(r"[≤<]=?\s*([0-9.]+)", s)
    if m:
        return np.nan, float(m.group(1))

    # 若字符串为空或非数字
    if s.lower() in ["yes", "no", "none", "nan", ""]:
        return np.nan, np.nan

    # === 1. 直接数值 ===
    if re.match(r"^-?\d+(\.\d+)?$", s):
        val = float(s)
        return val, val

    # === 2. 区间格式 (a-b, a~b, a–b, 含 d) ===
    m = re.match(r"^\s*(-?\d+(?:\.\d+)?)\s*[-–~]\s*(-?\d+(?:\.\d+)?)([a-zA-Z]*)\s*$", s)
    if m:
        lb = float(m.group(1))
        ub = float(m.group(2))
        if lb > ub:  # 自动调整方向
            lb, ub = ub, lb
        return lb, ub

    # === 3. ± 容差格式 (10 ± 2.5) ===
    m = re.match(r"^\s*(-?\d+(?:\.\d+)?)\s*[±]\s*(\d+(?:\.\d+)?)", s)
    if m:
        center, tol = map(float, m.groups())
        return center - tol, center + tol

    # === 4. 约等于格式 (≈ or ~= or ~X)
    m = re.match(r"^[≈~]\s*(-?\d+(?:\.\d+)?)", s)
    if m:
        val = float(m.group(1))
        return val, val

    # === 5. 大于 / 小于号 ===
    m = re.match(r"^[>]\s*(-?\d+(?:\.\d+)?)", s)
    if m:
        return float(m.group(1)), np.nan
    m = re.match(r"^[<]\s*(-?\d+(?:\.\d+)?)", s)
    if m:
        return np.nan, float(m.group(1))

    # === 6. 带 d 的情况 (2-5d, >7d)
    m = re.match(r"^(\d+)\s*[-–~]\s*(\d+)d$", s)
    if m:
        return float(m.group(1)), float(m.group(2))
    m = re.match(r"^>\s*(\d+)d$", s)
    if m:
        return float(m.group(1)), np.nan

    # === 7. fallback ===
    return np.nan, np.nan




In [349]:
item_nos = """
P4002
P4004
P4005
P4006
P4007
P4008
P4013
P4041
P4079
P4080
P4338
P4339
P4342
P4343
P4433
""".strip().split()

# item_nos = """
# P4002
# P4005
# P4006
# P4013
# P4041
# P4433
# P4007
# P4008
# P4079
# P4080
# """.strip().split()


item_nos = sorted(list(set(item_nos)))
print(f"total item no: {len(item_nos)}")

print("\n".join(item_nos))



# @All 今天和业务快速对齐，对分析范围有一些调整：
# 移除关注特征：4338，4339，4342，4343（在ASTM上的测试结果，数据较少，不再进行预测）
# 增加需要移除的产品属性：P4004（Removable force，业务经验和liner ## release force等4个材料参数紧密相关;GPT建议同时关注liner ## moisture* (preliminary) ## 、liner ## moisture ## %、liner ## anchorage on printed side等参数 )
# 产品的Total thickness, without liner相关属性：P4433和P4002融合为一个特征
# 在预测结果中，推荐的产品信息补充Test_method内容（现有产品匹配）


# 此外，在现有的材料属性中，增加新的因素：产品的total_thickness - Total thickness, without liner （P4004)、P4080
# 

total item no: 15
P4002
P4004
P4005
P4006
P4007
P4008
P4013
P4041
P4079
P4080
P4338
P4339
P4342
P4343
P4433


In [350]:

import pandas as pd

def load_product_spec_from_excel(file_path: str, sheet_name: str) -> pd.DataFrame:
    """
    Loads data from a specified sheet in an Excel file into a pandas DataFrame.

    This function is designed to handle a specific format where:
    - The header is located on the second row of the sheet.
    - The first column contains a merged cell, and its value needs to be
      propagated to all rows of the DataFrame.

    Args:
        file_path (str): The path to the Excel file.
        sheet_name (str): The name of the sheet to load.

    Returns:
        Optional[pd.DataFrame]: A pandas DataFrame containing the loaded and
                                cleaned data, or None if an error occurs.
    """

    # Read the Excel file, specifying that the header is the second row (index 1).
    # pandas automatically handles merged cells by assigning the value to the
    # top-left cell of the merge area and leaving others as NaN.
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=0)

    # Identify the name of the first column (e.g., 'Product Spec').
    first_column_name = df.columns[0]

    # Use forward-fill (ffill) to propagate the value from the merged cell
    # down through the entire column.
    df[first_column_name] = df[first_column_name].ffill()

    # Drop any rows that are completely empty, which might occur if there are
    # blank rows at the end of the sheet.
    df.dropna(how='all', inplace=True)

    return df



In [382]:
# excel文件
excel_file_path = "/Users/xgq/Library/CloudStorage/OneDrive-SharedLibraries-IndustrialMind.ai/IndustrialMind Product - Documents/50_Customers/Tesa德莎/PDA/30-Data Example/ForModeling/PD Accelerator Data List_20251104-updated.xlsx"

df_product = load_product_spec_from_excel(file_path=excel_file_path, sheet_name="Product Property")

print(
    "load df_product record num",
    len(df_product),
)
print("-" * 120)


# strip columns
df_product["Product Spec"] = df_product["Product Spec"].fillna("").str.strip()
df_product["Item No."] = df_product["Item No."].fillna("").str.strip()
df_product["Unit"] = df_product["Unit"].fillna("").str.strip()
df_product["Test Method"] = df_product["Test Method"].fillna("").str.strip()
df_product["Target Value"] = df_product["Target Value"].fillna("").str.strip()


# filter by item_nos
df_product = df_product[df_product["Item No."].isin(item_nos)].reset_index(drop=True)
print(
    "after filtering by item_nos, df_product record num",
    len(df_product),
    "\n",
    "-" * 120,
)

# dedup
df_product = df_product.drop_duplicates(subset=["Product Spec", "Item No."]).reset_index(drop=True)
print(
    "after dedup by Product Spec and Item No., df_product record num",
    len(df_product),
    "\n",
    "-" * 120
)
print()

# property key and value
df_product["property_key"] = df_product["Item No."].str.lower() + " ## " \
                            + df_product["Unit"].str.lower()  + " ## " \
                            + df_product["Test Method"].str.lower()

df_product["property_value"] = df_product["Target Value"]


# parse lb and ub
df_product[["lb", "ub"]] = df_product["property_value"].apply(
    lambda x: pd.Series(parse_lb_ub(str(x)))
)


# display preview
print(
    "product samples:\n",
    df_product.head(5).to_string(),
    "\n",
    "-" * 120,
)



# df_product: 记录了product的各项属性，包括Item No.，Unit，Test Method，Target Value，以及解析后的lb和ub。





load df_product record num 1225
------------------------------------------------------------------------------------------------------------------------
after filtering by item_nos, df_product record num 358 
 ------------------------------------------------------------------------------------------------------------------------
after dedup by Product Spec and Item No., df_product record num 349 
 ------------------------------------------------------------------------------------------------------------------------

product samples:
      Product Spec  No.                                      Item Description Item No.  Unit  Target Value         Test Method Test Type                         property_key property_value    lb    ub
0  62565-70000-57    1         Total weight after 1st coating, without liner    P4079  g/m²  37.00 ± 5.00            J0PM0005         I            p4079 ## g/m² ## j0pm0005   37.00 ± 5.00  32.0  42.0
1  62565-70000-57    2                           Total weig

In [391]:
# Item No stats

print(
    "total item no. number",
    len(df_product["Item No."].unique()),
    "\n",
    "-" * 120,
)
df_item_no = df_product["Item No."].value_counts().reset_index()
df_item_no = df_item_no.rename(columns={"count": "product count"})


print(
    "df_item_no preview:\n",
    df_item_no.head(len(df_item_no)).to_string(),
    "\n",
    "-" * 120,
)


df_item_no_name = df_product.groupby("Item No.")["Item Description"].first().reset_index()
print(
    "item no description mapping preview:\n", 
    df_item_no_name.head(len(df_item_no_name)).to_string(),
    "\n",
    "-" * 120
)



df_item_no = df_item_no.merge(df_item_no_name,
                              left_on="Item No.",
                              right_on="Item No.",
                            )

print(
    "item no name mapping preview:\n",
    df_item_no.head().to_string(),
    "\n",
    "-" * 120,
)

df_item_no.to_csv("data/item_no_name_mapping.csv", index=False, sep="|", encoding="utf-8")


# df_item_no记录了每个Item No.对应的产品数量，以及Item Description。


# 调整dataframe配置，不限制pandas 输出宽度
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(
    "item no preview:\n",
    df_item_no.sort_values(by="Item No.",).to_string(),
    "\n",
    "-" * 120,
)

# item no short names mapping
item_no_short_names = {
    "P4002": "Ttl_thick_wto_liner_4n",
    "P4005": "PA_Steel_open",
    "P4006": "PA_Steel_cover",
    "P4007": "PA_Steel_14d_open",
    "P4008": "PA_Steel_14d_cover",
    "P4013": "PA_PC",
    "P4041": "HP_Steel_open",
    "P4079": "Wgt_after_1st_coating_wto_liner",
    "P4080": "Wgt_wto_liner",
    "P4338": "PA_steel_inside",
    "P4339": "PA_steel_outside",
    "P4342": "PA_ASTM_open",
    "P4343": "PA_ASTM_cover",
    "P4433": "Ttl_thick_wto_liner_10n",
}




total item no. number 14 
 ------------------------------------------------------------------------------------------------------------------------
df_item_no preview:
    Item No.  product count
0     P4080             42
1     P4433             42
2     P4079             41
3     P4005             32
4     P4006             32
5     P4007             32
6     P4008             32
7     P4004             29
8     P4041             26
9     P4013             25
10    P4338              4
11    P4339              4
12    P4342              4
13    P4343              4 
 ------------------------------------------------------------------------------------------------------------------------
item no description mapping preview:
    Item No.                                         Item Description
0     P4004             Removable force of paper liner (180° method)
1     P4005             Peel adhesion/Steel, open side (180° method)
2     P4006          Peel adhesion/Steel, covered side (18

# Product building blocks
产品由何种材料组成

In [355]:
import pandas as pd
import openpyxl
from openpyxl.utils import column_index_from_string

def read_excel_with_merged_cells(file_path: str, sheet_name: str, merged_column: str) -> pd.DataFrame:
    """
    Reads an Excel file, handles merged cells, ignores empty rows, and filters out empty columns.

    Args:
    - file_path (str): The path to the Excel file.
    - sheet_name (str): The name of the sheet to read.
    - merged_column (str): The column letter with merged cells (e.g., 'A').

    Returns:
    - pd.DataFrame: A DataFrame with the merged cells correctly filled.
    """
    try:
        workbook = openpyxl.load_workbook(file_path, data_only=True)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return pd.DataFrame()

    if sheet_name not in workbook.sheetnames:
        print(f"Error: Sheet '{sheet_name}' not found in the Excel file.")
        return pd.DataFrame()
        
    sheet = workbook[sheet_name]
    
    # Process header: strip whitespace and identify non-empty columns
    header_row = sheet[1]
    header = []
    valid_column_indices = []
    for i, cell in enumerate(header_row):
        if cell.value and str(cell.value).strip():
            header.append(str(cell.value).strip())
            valid_column_indices.append(i)

    merged_col_idx_in_original = column_index_from_string(merged_column) - 1
    
    # Find the new index of the merged column in the filtered set of columns
    try:
        merged_col_idx = valid_column_indices.index(merged_col_idx_in_original)
    except ValueError:
        print(f"Error: Merged column '{merged_column}' is empty or not found in the header.")
        return pd.DataFrame(columns=header)

    data = []
    current_merged_value = None

    for row_cells in sheet.iter_rows(min_row=2):
        
        # Check if the row is entirely epty
        if all(cell.value is None for cell in row_cells):
            continue

        # Filter row data to include only valid columns
        record = [row_cells[i].value for i in valid_column_indices]

        merged_cell_value = record[merged_col_idx]
        
        if merged_cell_value is not None:
            current_merged_value = merged_cell_value
        else:
            # If merged_cell.value is None, it means it's part of a merged block,
            # and we should use the last seen value (current_merged_value).
            record[merged_col_idx] = current_merged_value
        
        data.append(record)

    df = pd.DataFrame(data, columns=header)
    return df

In [392]:
# excel file path
excel_file_path = "/Users/xgq/Library/CloudStorage/OneDrive-SharedLibraries-IndustrialMind.ai/IndustrialMind Product - Documents/50_Customers/Tesa德莎/PDA/30-Data Example/ForModeling/PD Accelerator Data List_20251104-updated.xlsx"
sheet_name = "Building Blocks" 
merged_col_name = "A"


# load data
df_build_blocks = read_excel_with_merged_cells(
    file_path=excel_file_path,
    sheet_name=sheet_name,
    merged_column=merged_col_name
)
print(
    "original df_build_blocks record num\n",
    len(df_build_blocks),
    "\n",
    "-" * 120
)

# strip columns
df_build_blocks["Product Specification"] = df_build_blocks["Product Specification"].fillna("").str.strip()
df_build_blocks["NART"] = df_build_blocks["NART"].fillna("").str.strip()
df_build_blocks["type"] = df_build_blocks["type"].fillna("").str.strip()


# rm leading "E-FER-" from "Product Specification"
df_build_blocks["Product Specification"] = df_build_blocks["Product Specification"].str.replace(r"^E-FER-", "", regex=True)


# duplication detection
ret = df_build_blocks.groupby(["Product Specification", "NART", "type"]).size().reset_index(name="counts").query("counts > 1")

print(
    "duplication detection result:\n",
    ret.head(5).to_string(),
    "\n",
    "-" * 120
)


# dedup
df_build_blocks = df_build_blocks.drop_duplicates(subset=["Product Specification", "NART", "type"]).reset_index(drop=True)
print(
    "after dedup by, df_build_blocks record num\n",
    len(df_build_blocks),
    "\n",
    "-" * 120
)



print(
    "df_build_blocks preview:\n",
    df_build_blocks.head().to_string(),
    "\n",
    "-" * 120
)


# df_build_blocks： 记录product 有哪些 building blocks 组成


original df_build_blocks record num
 258 
 ------------------------------------------------------------------------------------------------------------------------
duplication detection result:
    Product Specification            NART       type  counts
21        66905-70000-40  14064-80000-80  adhensive       2
24        66910-70000-40  14064-80000-80  adhensive       2
28        66930-70000-40  14174-80000-00  adhensive       2
32        66932-70000-40  14174-80000-00  adhensive       2
38        66934-70000-40  14174-80000-00  adhensive       2 
 ------------------------------------------------------------------------------------------------------------------------
after dedup by, df_build_blocks record num
 209 
 ------------------------------------------------------------------------------------------------------------------------
df_build_blocks preview:
   Product Specification                   block identification            NART       type  OR notes if_has
0        62565-700

# Material

In [357]:
def process_property_col(s: str) -> str:
    if s is None or len(s) == 0:
        return ""
    s = s.strip()

    # remove trailing *
    s = s.rstrip("*")

    # remove trailing ¹
    s = s.rstrip("¹")

    # substitute `U+03BC` with `U+00B5`
    s = s.replace('\u03bc', '\u00B5')

    return s.strip()


## Adhesive property

In [358]:
import pandas as pd
import openpyxl
from openpyxl.utils import column_index_from_string

def read_material_property_sheet(file_path: str, sheet_name: str, merged_columns: list) -> pd.DataFrame:
    """
    Reads an Excel sheet with multiple merged columns, handles merged cells correctly,
    and ignores empty columns.

    Args:
    - file_path (str): The path to the Excel file.
    - sheet_name (str): The name of the sheet to read.
    - merged_columns (list): A list of column letters with merged cells (e.g., ['A', 'B']).

    Returns:
    - pd.DataFrame: A DataFrame with the merged cells correctly filled.
    """
    try:
        workbook = openpyxl.load_workbook(file_path, data_only=True)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return pd.DataFrame()

    if sheet_name not in workbook.sheetnames:
        print(f"Error: Sheet '{sheet_name}' not found in the Excel file.")
        return pd.DataFrame()
        
    sheet = workbook[sheet_name]
    
    # Process header: strip whitespace and identify non-empty columns
    header_row = sheet[1]
    header = []
    valid_column_indices = []
    for i, cell in enumerate(header_row):
        if cell.value and str(cell.value).strip():
            header.append(str(cell.value).strip())
            valid_column_indices.append(i)

    original_merged_col_indices = [column_index_from_string(col) - 1 for col in merged_columns]
    
    current_merged_values = [None] * len(merged_columns)
    data = []

    for row_cells in sheet.iter_rows(min_row=2):
        
        # Check if the row is entirely empty and skip it
        if all(cell.value is None for cell in row_cells):
            continue

        # Filter row data to include only valid columns
        record = [row_cells[i].value for i in valid_column_indices]

        # Handle merged columns based on their original positions
        for i, original_col_idx in enumerate(original_merged_col_indices):
            if original_col_idx in valid_column_indices:
                # Find the new index in the filtered record
                new_idx = valid_column_indices.index(original_col_idx)
                cell_value = record[new_idx]
                
                if cell_value is not None:
                    current_merged_values[i] = cell_value
                else:
                    record[new_idx] = current_merged_values[i]
        
        data.append(record)

    df = pd.DataFrame(data, columns=header)
    return df

In [393]:
# file path
excel_file_path = "/Users/xgq/Library/CloudStorage/OneDrive-SharedLibraries-IndustrialMind.ai/IndustrialMind Product - Documents/50_Customers/Tesa德莎/PDA/30-Data Example/ForModeling/PD Accelerator Data List_20251104-updated.xlsx"
sheet_name = "Adhesive Property"
merged_cols = ["A"] 


# load data
df_adhesive = read_material_property_sheet(
    file_path=excel_file_path,
    sheet_name=sheet_name,
    merged_columns=merged_cols
)

# strip columns
df_adhesive["Adhesive"] = df_adhesive["Adhesive"].fillna("").str.strip()

df_adhesive["Property"] = df_adhesive["Property"].apply(process_property_col)
df_adhesive["Condition"] = df_adhesive["Condition"].apply(process_property_col)


# drop dups
print(
    "load df_adhesive record num\n",
    len(df_adhesive),
    "\n",
    "-" * 120
)

df_adhesive = df_adhesive.drop_duplicates(["Adhesive", "Property", "Condition", "Value"])
print(
    "after dropping duplicates, adhesive property record num\n",
    len(df_adhesive),
    "\n",
    "-" * 120
)


# add NART column, property_key and property_value
df_adhesive["type"] = "adhesive"
df_adhesive["NART"] = df_adhesive["Adhesive"]

df_adhesive["property_key"] = "adhesive ## " + df_adhesive["Property"].str.lower() + " ## " \
                            + df_adhesive["Condition"].str.lower()

df_adhesive["property_value"] = df_adhesive["Value"]

# parse lb and ub
df_adhesive[["lb", "ub"]] = df_adhesive["property_value"].apply(
    lambda x: pd.Series(parse_lb_ub(str(x)))
)


print(
    "df_adhesive preview:\n",
    df_adhesive.head().to_string(),
    "\n",
    "-" * 120
)


load df_adhesive record num
 105 
 ------------------------------------------------------------------------------------------------------------------------
after dropping duplicates, adhesive property record num
 105 
 ------------------------------------------------------------------------------------------------------------------------
df_adhesive preview:
          Adhesive Adhesive-Name              Property     Condition   Value      type            NART                                  property_key property_value       lb   ub
0  14174-80000-00  Cello 33.146  Peel Adhesion (N/cm)           SUS     7.1  adhesive  14174-80000-00       adhesive ## peel adhesion (n/cm) ## sus            7.1      7.1  7.1
1  14174-80000-00          None  Peel Adhesion (N/cm)            PC     8.3  adhesive  14174-80000-00        adhesive ## peel adhesion (n/cm) ## pc            8.3      8.3  8.3
2  14174-80000-00          None  Holding Power (mins)            RT  >10000  adhesive  14174-80000-00      

In [360]:
print(
    "total property number\n",
    df_adhesive["property_key"].unique().shape[0]
)

total property number
 18


## Liner property

In [361]:
# excel file path
excel_file_path = "/Users/xgq/Library/CloudStorage/OneDrive-SharedLibraries-IndustrialMind.ai/IndustrialMind Product - Documents/50_Customers/Tesa德莎/PDA/30-Data Example/ForModeling/PD Accelerator Data List_20251104-updated.xlsx"
sheet_name = "Liner Property"
merged_cols = ["A"]

# Reusing the function from 'Adhesive Property' section
df_liner = read_material_property_sheet(
    file_path=excel_file_path,
    sheet_name=sheet_name,
    merged_columns=merged_cols
)
print(
    "load df_liner record num",
    len(df_liner),
)
print("-" * 120)


# strip columns
df_liner["Liner"] = df_liner["Liner"].fillna("").str.strip()
df_liner["Test Methods"] = df_liner["Test Methods"].fillna("").str.strip()
df_liner["Limits / Requirements"] = df_liner["Limits / Requirements"].fillna("").str.strip()

df_liner["Description"] = df_liner["Description"].apply(process_property_col)
df_liner["Units"] = df_liner["Units"].apply(process_property_col)


# drop dups
df_liner = df_liner.drop_duplicates(["Liner", "Description", "Units", "Test Methods", "Limits / Requirements"])
print(
    "after dropping duplicates, liner property record num",
    len(df_liner),
)
print("-" * 120)


# material NART
df_liner["type"] = "liner"
df_liner["NART"] = df_liner["Liner"]


# property_key and property_value
df_liner["property_key"] = "liner ## " + df_liner["Description"].str.lower() + " ## " \
                         + df_liner["Units"].str.lower()

df_liner["property_value"] = df_liner["Limits / Requirements"]

# parse lb and ub
df_liner[["lb", "ub"]] = df_liner["property_value"].apply(
    lambda x: pd.Series(parse_lb_ub(str(x)))
)

df_liner.to_csv('data/df_liner.csv', index=False)



print(
    "liner preview\n",
    df_liner.head(5).to_string()
)
print("-" * 120)



load df_liner record num 125
------------------------------------------------------------------------------------------------------------------------
after dropping duplicates, liner property record num 125
------------------------------------------------------------------------------------------------------------------------
liner preview
             Liner Serial Number                                        Description                              Limits / Requirements Units                               Test Methods   type            NART                                                    property_key                                     property_value    lb    ub
0  22857-90000-00         1.1.1                                             Colour    White, printed with tesa Logo FINAT TLMI-No. 16        JOPM0041 comparison with reference sample  liner  22857-90000-00                                             liner ## colour ##     White, printed with tesa Logo FINAT TLMI-No. 16   N

In [362]:
print(
    "Total liner property number",
    df_liner["property_key"].unique().shape[0]
)

Total liner property number 44


## Backing property

In [394]:
# excel file path
excel_file_path = "/Users/xgq/Library/CloudStorage/OneDrive-SharedLibraries-IndustrialMind.ai/IndustrialMind Product - Documents/50_Customers/Tesa德莎/PDA/30-Data Example/ForModeling/PD Accelerator Data List_20251104-updated.xlsx"
sheet_name = "Backing Property"
merged_cols = ["A"]

# load data
# Reusing the function from 'Adhesive Property' section
df_backing = read_material_property_sheet(
    file_path=excel_file_path,
    sheet_name=sheet_name,
    merged_columns=merged_cols
)
print(
    "load df_backing record num",
    len(df_backing),
    "\n",
    "-" * 120
)


# strip columns
df_backing["Backing"] = df_backing["Backing"].fillna("").str.strip()
df_backing["tesa + DIN/ISO Standard"] = df_backing["tesa + DIN/ISO Standard"].fillna("").str.strip()
df_backing["Test Figures / Tolerances"] = df_backing["Test Figures / Tolerances"].fillna("").str.strip()


df_backing["Property"] = df_backing["Property"].apply(process_property_col)


# drop dups
df_backing = df_backing.drop_duplicates(["Backing", "Property", "tesa + DIN/ISO Standard", "Test Figures / Tolerances"])
print(
    "after dropping duplicates, backing property record num",
    len(df_backing),
    "\n",
    "-" * 120
)


# NART
df_backing["type"] = "backing"
df_backing["NART"] = df_backing["Backing"]


# property_key and property_value
df_backing["property_key"] = "backing ## " + df_backing["Property"].str.lower()
df_backing["property_value"] = df_backing["Test Figures / Tolerances"]


# parse lb and ub
df_backing[["lb", "ub"]] = df_backing["property_value"].apply(
    lambda x: pd.Series(parse_lb_ub(str(x)))
)




print(
    "backing preview\n",
    df_backing.head(5).to_string(),
    "\n",
    "-" * 120
)


load df_backing record num 70 
 ------------------------------------------------------------------------------------------------------------------------
after dropping duplicates, backing property record num 70 
 ------------------------------------------------------------------------------------------------------------------------
backing preview
           Backing                Property          Test Figures / Tolerances                                          tesa + DIN/ISO Standard     type            NART                       property_key                     property_value    lb     ub
0  01446-90016-81               Thickness                          12 ± 1 μm                                                         JOPMC002  backing  01446-90016-81               backing ## thickness                          12 ± 1 μm  11.0   13.0
1  01446-90016-81    Weight per unit area                    16,8 ± 1,7 g/m²                                        JOPM0005 (100 cm² sample)  backin

In [395]:
df_backing["property_key"].value_counts().sort_index()

property_key
backing ##                                         1
backing ## antimony content                        1
backing ## colour difference value δe cmc          1
backing ## elongation at break cd                  7
backing ## elongation at break md                  8
backing ## elongation at break td                  1
backing ## shrinkage cd                            7
backing ## shrinkage md                            8
backing ## shrinkage td                            1
backing ## tensile strength at 5% elongation md    1
backing ## tensile strength cd                     7
backing ## tensile strength md                     8
backing ## tensile strength td                     1
backing ## thickness                               8
backing ## transparency                            1
backing ## transparency vlt (550 nm)               1
backing ## weight per unit area                    8
Name: count, dtype: int64

## 查找缺失adhesive, backing, liner的records

In [396]:
# 过滤df_building_blocks 中type == "adhesive" 的记录，然后join df_adhesive, 过滤join失败的记录

df_missing_adhesive = df_build_blocks[df_build_blocks["type"].str.lower() == "adhesive"][["NART", "Product Specification"]].merge(
    df_adhesive,
    how="left",
    left_on=["NART"],
    right_on=["NART"],
    indicator=True
)

df_missing_adhesive = df_missing_adhesive[df_missing_adhesive["_merge"] == "left_only"]
print(
    "missing adhesive records after join:\n",
    df_missing_adhesive["NART"].value_counts().to_string()
)
print("-" * 120)

missing adhesive records after join:
 Series([], )
------------------------------------------------------------------------------------------------------------------------


In [397]:
# 过滤df_building_blocks 中type == "backing" 的记录，然后join df_backing, 过滤join失败的记录

df_missing_backing = df_build_blocks[df_build_blocks["type"].str.lower() == "backing"][["NART", "Product Specification"]].merge(
    df_backing,
    how="left",
    left_on=["NART"],
    right_on=["NART"],
    indicator=True  
)

df_missing_backing = df_missing_backing[df_missing_backing["_merge"] == "left_only"]

print(
    "missing backing records after join:\n",
    df_missing_backing["NART"].value_counts().to_string()
)
print("-" * 120)

missing backing records after join:
 NART
20466-90001-00    5
20808-90000-00    3
20491-90000-80    3
20599-90001-00    3
20153-90000-00    1
20654-90000-80    1
------------------------------------------------------------------------------------------------------------------------


In [398]:
# 过滤df_building_blocks 中type == "liner" 的记录，然后join df_liner, 过滤join失败的记录

df_missing_liner = df_build_blocks[df_build_blocks["type"].str.lower() == "liner"][["NART", "Product Specification"]].merge(
    df_liner,
    how="left",
    left_on=["NART"],
    right_on=["NART"],
    indicator=True
)

df_missing_liner = df_missing_liner[df_missing_liner["_merge"] == "left_only"]
print(
    "missing liner records after join:\n",
    df_missing_liner["NART"].value_counts().to_string()
)
print("-" * 120)



missing liner records after join:
 NART
20057-90002-00    5
20779-90000-80    2
21103-90000-00    2
21104-90000-00    2
22957-90000-80    2
22895-98001-10    2
22955-90000-80    2
------------------------------------------------------------------------------------------------------------------------


## Property name to id

In [400]:
import json


# stack all materials vertically
df_materials = pd.concat(
    [
        df_adhesive[["type", "NART", "property_key", "property_value", "lb", "ub"]], 
        df_liner[["type", "NART", "property_key", "property_value", "lb", "ub"]], 
        df_backing[["type", "NART", "property_key", "property_value", "lb", "ub"]],
    ],
    ignore_index=True
)

print(
    "df_materials preview\n",
    df_materials.head(5).to_string(),
    "\n",
    "-" * 120
)


df_materials.to_csv('data/df_materials.csv', index=False)
print(
    "Saved combined materials data to data/df_materials.csv",
    "\n",
    "-" * 120
)


# distinct NART number
print(
    "total distinct NART number",
    len(df_materials['NART'].unique()),
    "\n",
    "-" * 120
)


# property names
property_names = df_materials["property_key"].value_counts().index.values.tolist()
property_names = sorted(property_names)
print(
    "total property names number",
    len(property_names),
    "\n",
    "-" * 120
)


# property name to index
property_name_to_id = {
    name: idx for idx, name in enumerate(property_names)
}

with open("data/property_name_to_id.json", "w") as f:
    json.dump(property_name_to_id, f, indent=4, ensure_ascii=False)
    print(
        "Saved property_name_to_id to data/property_name_to_id.json",
        "\n",
        "-" * 120
    )


property_id_to_name = {
    idx: name for name, idx in property_name_to_id.items()
}


df_materials preview
        type            NART                                  property_key property_value       lb   ub
0  adhesive  14174-80000-00       adhesive ## peel adhesion (n/cm) ## sus            7.1      7.1  7.1
1  adhesive  14174-80000-00        adhesive ## peel adhesion (n/cm) ## pc            8.3      8.3  8.3
2  adhesive  14174-80000-00        adhesive ## holding power (mins) ## rt         >10000  10000.0  NaN
3  adhesive  14174-80000-00              adhesive ## edge taping ## 60/90            >7d      7.0  NaN
4  adhesive  14174-80000-00  adhesive ## static peel (mm) ## 50g,70℃ 24hr              1      1.0  1.0 
 ------------------------------------------------------------------------------------------------------------------------
Saved combined materials data to data/df_materials.csv 
 ------------------------------------------------------------------------------------------------------------------------
total distinct NART number 40 
 ---------------------------

# Product label

In [369]:
import pandas as pd

def read_excel_product_specs(file_path: str, sheet_name: str) -> pd.DataFrame:
    """
    Reads product specification data from an Excel sheet with a specific 
    two-level, partially merged header structure.

    Args:
        file_path (str): The path to the Excel file.
        sheet_name (str): The name of the sheet containing the data.

    Returns:
        pd.DataFrame: A pandas DataFrame with a cleaned, multi-level column index 
                      that represents the header structure.
    """
    # Read the Excel file, specifying that the first two rows (0 and 1) 
    # should be combined into a MultiIndex header.
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=[0, 1])

    # The raw MultiIndex will contain placeholder names like 'Unnamed:...'
    # for empty cells in the header rows. We need to clean these up.
    
    raw_columns = df.columns.to_list()
    cleaned_columns = []

    # Process each tuple in the raw MultiIndex
    for top_level, bottom_level in raw_columns:
        
        # Clean the top-level header. If it contains 'Unnamed', 
        # it corresponds to a header cell that was empty.
        # We replace it with an empty string.
        cleaned_top = '' if 'Unnamed' in str(top_level) else top_level
        
        # Clean the bottom-level header. The first column's bottom header
        # is also a placeholder we should remove.
        cleaned_bottom = '' if 'Unnamed' in str(bottom_level) else bottom_level
        
        cleaned_columns.append((cleaned_top, cleaned_bottom))

    # Assign the cleaned list of tuples back to the DataFrame's columns,
    # creating a proper MultiIndex.
    df.columns = pd.MultiIndex.from_tuples(cleaned_columns)

    # To make the first column easier to access, we can rename it.
    # The original header is ('Product Specification', '').
    df = df.rename(columns={
        ('Product Specification', ''): ('', 'Product Specification')
    })

    return df




In [401]:
# file path
excel_file_path = "/Users/xgq/Library/CloudStorage/OneDrive-SharedLibraries-IndustrialMind.ai/IndustrialMind Product - Documents/50_Customers/Tesa德莎/PDA/30-Data Example/ForModeling/PD Accelerator Data List_20251104-updated.xlsx"
sheet_name = "Product Label"


df_product_label = read_excel_product_specs(file_path=excel_file_path, sheet_name=sheet_name)
print(
    "load df_product_label record num",
    len(df_product_label),
    "\n",
    "-" * 120
)


# reset columns
columns = []
for col in df_product_label.columns:
    if col[0] and col[1]:
        new_col = col[1]
    elif col[0]:
        new_col = col[0]
    else:
        new_col = col[1]
    columns.append(new_col.strip())

df_product_label.columns = columns


# strip columns
df_product_label["Product Specification"] = df_product_label["Product Specification"].fillna("").str.strip()
df_product_label["L1"] = df_product_label["L1"].fillna("").str.strip()
df_product_label["L2"] = df_product_label["L2"].fillna("").str.strip()

# drop dups
df_product_label = df_product_label.drop_duplicates(["Product Specification", "L1", "L2"])


# remove FER- prefix from Product Specification
df_product_label["Product Specification"] = df_product_label["Product Specification"].fillna("").str.replace(r"^(E-)?FER-", "", regex=True)


print(
    "After dedup, df_product label record num",
    len(df_product_label),
    "\n",
    "-" * 120
)


print(
    "product label preview:\n",
    df_product_label.head(5).to_string(),
    "\n",
    "-" * 120
)


load df_product_label record num 20 
 ------------------------------------------------------------------------------------------------------------------------
After dedup, df_product label record num 20 
 ------------------------------------------------------------------------------------------------------------------------
product label preview:
   Product Specification              L1                L2             L3             L4             L5            L6   L7  L8
0        66930-70000-40  Anti-repulsion     High cohesion  quick bonding   FPC mounting            NaN           NaN  NaN NaN
1        68537-70000-40  Anti-repulsion  General mounting  quick bonding   FPC mounting            NaN           NaN  NaN NaN
2        68705-70000-67  Anti-repulsion     High cohesion  Quick bonding       Transfer  High cosmetic  FPC mounting  NaN NaN
3        68825-70000-58  Anti-repulsion          Bio-base  High cohesion  High cosmetic   FPC mounting           NaN  NaN NaN
4        68560-70000

# Build dataset

## product_spec

In [403]:
df_product_spec = df_product[["Product Spec"]].drop_duplicates().reset_index(drop=True).copy()

print("total product num", len(df_product_spec))


print(
    "product spec sample records:\n",
    df_product_spec["Product Spec"].head(5).to_string(),
    "\n",
    "-" * 120
)



total product num 42
product spec sample records:
 0    62565-70000-57
1    62573-70000-55
2    66505-70000-55
3    66509-70000-57
4    66905-70000-40 
 ------------------------------------------------------------------------------------------------------------------------


## 扩展item no

In [404]:
# 扩展item no

item_no_counts = df_product["Item No."].value_counts()

df_product_item_no = df_product_spec.copy()

def mean_of_lb_and_ub(lb: float, ub: float) -> float | None:
    if not pd.isna(lb) and not pd.isna(ub):
        return (lb + ub) / 2
    elif not pd.isna(lb):
        return lb
    elif not pd.isna(ub):
        return ub
    else:
        return None


# join item no
for item_no, count in item_no_counts.items():
    print(
        f"processing item no {item_no} with count {count}",
        "\n",
        "-" * 120
    )

    df_product_item_no = df_product_item_no.merge(
        df_product[df_product["Item No."] == item_no][["Product Spec", "property_value", "lb", "ub"]],
        left_on="Product Spec",
        right_on="Product Spec",
        how="left",
    )

    # rename columns
    df_product_item_no = df_product_item_no.rename(columns={
        "property_value": f"{item_no}_value",
        "lb": f"{item_no}_lb",
        "ub": f"{item_no}_ub",
    })

    # target value
    item_simple_name = item_no_short_names.get(item_no, "")
    df_product_item_no[f"{item_no}_{item_simple_name}_target_value"] = df_product_item_no.apply(
        lambda row: mean_of_lb_and_ub(row[f"{item_no}_lb"], row[f"{item_no}_ub"]),
        axis=1
    )

    print(
        "after joining, df_product_item_no record num",
        len(df_product_item_no),
        "\n",
        "-" * 120
    )


print(
    "final df_product_item_no preview:\n",
    df_product_item_no.head(5).to_string(),
    "\n",
    "-" * 120
)


processing item no P4080 with count 42 
 ------------------------------------------------------------------------------------------------------------------------
after joining, df_product_item_no record num 42 
 ------------------------------------------------------------------------------------------------------------------------
processing item no P4433 with count 42 
 ------------------------------------------------------------------------------------------------------------------------
after joining, df_product_item_no record num 42 
 ------------------------------------------------------------------------------------------------------------------------
processing item no P4079 with count 41 
 ------------------------------------------------------------------------------------------------------------------------
after joining, df_product_item_no record num 42 
 ------------------------------------------------------------------------------------------------------------------------
p

## 扩展 building NART

In [405]:
# 从df_product_spec出发, join df_build_blocks, 依次 获取adhesive, liner, backing 的NART
df_product_narts = df_product_item_no.copy()


# join adhesive NART
df_product_narts = pd.merge(
    df_product_narts,
    df_build_blocks[df_build_blocks["type"] == "adhensive"][["Product Specification", "NART"]],
    left_on="Product Spec",
    right_on="Product Specification",
    how='inner'
).rename(columns={"NART": "Adhesive_NART"})\
    .drop(columns=['Product Specification'])

# 去重
df_product_narts = df_product_narts.drop_duplicates(["Product Spec", "Adhesive_NART"])
print("after joining adhensive, product num", len(df_product_narts), "\n", "-" * 120)


# join liner NART
df_product_narts = pd.merge(
    df_product_narts,
    df_build_blocks[df_build_blocks["type"] == "liner"][['Product Specification', 'NART']],
    left_on="Product Spec",
    right_on='Product Specification',
    how='inner'
).rename(columns={"NART": "Liner_NART"})\
    .drop(columns=['Product Specification'])

# 去重
df_product_narts = df_product_narts.drop_duplicates(["Product Spec", "Adhesive_NART", "Liner_NART"])
print("after joining liner, product number", len(df_product_narts), "\n", "-" * 120)


# join backing NART
df_product_narts = pd.merge(
    df_product_narts,
    df_build_blocks[df_build_blocks["type"] == "backing"][['Product Specification', 'NART']],
    left_on="Product Spec",
    right_on='Product Specification',
    how='inner'
).rename(columns={"NART": "Backing_NART"})\
    .drop(columns=['Product Specification'])


# 去重
df_product_narts = df_product_narts.drop_duplicates(["Product Spec", "Adhesive_NART", "Liner_NART", "Backing_NART"])


print("after joining backing, product num", len(df_product_narts), "\n", "-" * 120)


print(
    "final df_product_narts preview:\n",
    df_product_narts.head(5).to_string(),
    "\n",
    "-" * 120
)

after joining adhensive, product num 65 
 ------------------------------------------------------------------------------------------------------------------------
after joining liner, product number 136 
 ------------------------------------------------------------------------------------------------------------------------
after joining backing, product num 169 
 ------------------------------------------------------------------------------------------------------------------------
final df_product_narts preview:
      Product Spec   P4080_value  P4080_lb  P4080_ub  P4080_Wgt_wto_liner_target_value P4433_value  P4433_lb  P4433_ub  P4433_Ttl_thick_wto_liner_10n_target_value   P4079_value  P4079_lb  P4079_ub  P4079_Wgt_after_1st_coating_wto_liner_target_value P4005_value  P4005_lb  P4005_ub  P4005_PA_Steel_open_target_value P4006_value  P4006_lb  P4006_ub  P4006_PA_Steel_cover_target_value P4007_value  P4007_lb  P4007_ub  P4007_PA_Steel_14d_open_target_value P4008_value  P4008_lb  P4008

## 扩展 product label

In [407]:

df_product_narts_label = df_product_narts.copy()


# join product label
df_product_narts_label = df_product_narts_label.merge(
    df_product_label[["Product Specification", "L1", "L2"]],
    left_on="Product Spec",
    right_on="Product Specification",
    how="left"
).drop(columns=["Product Specification"])


print(
    "after expanding product property, record num ",
    len(df_product_narts_label),
    "\n",
    "-" * 120
)


print("product buidling narts preview:", "\n", 
      df_product_narts_label.head().to_string(), 
      "\n", 
      "-" * 120,
)



after expanding product property, record num  169 
 ------------------------------------------------------------------------------------------------------------------------
product buidling narts preview: 
      Product Spec   P4080_value  P4080_lb  P4080_ub  P4080_Wgt_wto_liner_target_value P4433_value  P4433_lb  P4433_ub  P4433_Ttl_thick_wto_liner_10n_target_value   P4079_value  P4079_lb  P4079_ub  P4079_Wgt_after_1st_coating_wto_liner_target_value P4005_value  P4005_lb  P4005_ub  P4005_PA_Steel_open_target_value P4006_value  P4006_lb  P4006_ub  P4006_PA_Steel_cover_target_value P4007_value  P4007_lb  P4007_ub  P4007_PA_Steel_14d_open_target_value P4008_value  P4008_lb  P4008_ub  P4008_PA_Steel_14d_cover_target_value P4004_value  P4004_lb  P4004_ub  P4004__target_value P4041_value  P4041_lb  P4041_ub  P4041_HP_Steel_open_target_value P4013_value  P4013_lb  P4013_ub  P4013_PA_PC_target_value P4338_value  P4338_lb  P4338_ub  P4338_PA_steel_inside_target_value P4339_value  P4339_lb  P43

## 扩展材料属性

In [ ]:
# join material properrty


all_product_rows = []

for idx, row in df_product_narts_label.iterrows():
    row_dict = row.to_dict()

    feature_vec = [None] * len(property_name_to_id)

    # adhesive properties
    adhesive_nart = row_dict["Adhesive_NART"]
    adhensive_props = df_adhesive[df_adhesive["NART"] == adhesive_nart]
    if adhensive_props.empty:
        print(f"No adhesive found for NART {adhesive_nart}, skip", "\n", "-" * 120)
        continue

    for _, prop_row in adhensive_props.iterrows():
        value = mean_of_lb_and_ub(float(prop_row["lb"]), float(prop_row["ub"]))

        property_key = prop_row["property_key"]
        property_id = property_name_to_id.get(property_key, None)
        if property_id is None:
            print("warning: property_id not found:", property_key, "\n", "-" * 120)
            continue
        if property_id >= len(feature_vec):
            print("warning: property_id out of range:", property_id, property_key, "\n", "-" * 120)
            continue
        feature_vec[property_id] = value

    
    # liner properties
    liner_nart = row_dict["Liner_NART"]
    liner_props = df_liner[df_liner["NART"] == liner_nart]
    for _, prop_row in liner_props.iterrows():
        value = mean_of_lb_and_ub(float(prop_row["lb"]), float(prop_row["ub"]))

        property_key = prop_row["property_key"]
        property_id = property_name_to_id.get(property_key, None)
        if property_id is None:
            print("warning: property_id not found:", property_key, "\n", "-" * 120)
            continue
        if property_id >= len(feature_vec):
            print("warning: property_id out of range:", property_id, property_key, "\n", "-" * 120)
            continue
        feature_vec[property_id] = value

    
    # backing properties
    backing_nart = row_dict["Backing_NART"]
    backing_props = df_backing[df_backing["NART"] == backing_nart]
    for _, prop_row in backing_props.iterrows():
        value = mean_of_lb_and_ub(float(prop_row["lb"]), float(prop_row["ub"]))

        property_key = prop_row["property_key"]
        property_id = property_name_to_id.get(property_key, None)
        if property_id is None:
            print("warning: property_id not found:", property_key, "\n", "-" * 120)
            continue
        if property_id >= len(feature_vec):
            print("warning: property_id out of range:", property_id, property_key, "\n", "-" * 120)
            continue
        feature_vec[property_id] = value

    
    for idx, value in enumerate(feature_vec):
        # python dict key is order kept, so feature column won't be interleaved
        feature_name = "feature_" + property_id_to_name[idx]
        row_dict[feature_name] = value

    all_product_rows.append(row_dict)

    
    
print(
    "finish joining nart property", 
    "\n",
    "-" * 120
)


df_product_features = pd.DataFrame(all_product_rows)
print("Total product sample num", len(df_product_features), "\n", "-" * 120)



df_product_features.to_csv("data/df_product_features.csv", index=False, encoding="utf-8")
df_product_features.to_excel("data/df_product_features.xlsx")


print("product feature preview:\n", df_product_features.head().to_string(), "\n", "-" * 120)




No adhesive found for NART 13841-60000-81, skip 
 ------------------------------------------------------------------------------------------------------------------------
No adhesive found for NART 13841-60000-81, skip 
 ------------------------------------------------------------------------------------------------------------------------
No adhesive found for NART 13740-70000-92, skip 
 ------------------------------------------------------------------------------------------------------------------------
No adhesive found for NART 13740-70000-92, skip 
 ------------------------------------------------------------------------------------------------------------------------
No adhesive found for NART 13740-70000-92, skip 
 ------------------------------------------------------------------------------------------------------------------------
No adhesive found for NART 13740-70000-92, skip 
 -----------------------------------------------------------------------------------------------

In [376]:
# # 过滤 df_product_features 中 feature_backing ## thickness 为空的Product Spec	Adhesive_NART	Liner_NART	Backing_NART 
# df_filtered = df_product_features[df_product_features["feature_backing ## thickness"].isna()]
# print(
#     "After filtering feature_backing ## thickness not na, total product num",
#     len(df_filtered)
# )

# df_filtered[[
#     "Product Spec",
#     "Backing_NART",
#     "feature_backing ## thickness"
# ]].to_csv("data/data.csv", index=False, encoding="utf-8")
